# Using AutoML Text classification to predict Happy Moments in Text

Original Notebook from Google Cloud: [GCP Github](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb)

## Overview

The purpose of this project is to use autoML text to classify the text sentiment based on the labels shared

### Objective

In this tutorial, you learn how to use `AutoML` to train a text classification model.

This tutorial uses the following Google Cloud ML services:

- `AutoML Training`
- `Vertex AI Model` resource

The steps performed include:

* Create a `Vertex AI Dataset`.
* Train an `AutoML` text classification `Model` resource.
* Obtain the evaluation metrics for the `Model` resource.
* Create an `Endpoint` resource.
* Deploy the `Model` resource to the `Endpoint` resource.
* Make an online prediction
* Make a batch prediction

#### Execute notebook in Colab
<a href="https://colab.sandbox.google.com/github/paulycloud/ml_portfolio/blob/main/01_AutoML/02_text_classification/01_happyMoments_dataset/index.ipynb"> 
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
</a>

### Dataset

In this notebook, you use the [Happy Moments](https://www.kaggle.com/ritresearch/happydb). sample dataset to train a model. The resulting model classifies happy moments into categores that reflect the causes of happiness. 

### Install additional packages

Install the following packages for executing this notebook.

In [10]:
import os
import pandas as pd

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'

USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform google-cloud-storage jsonlines -q

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Project Variables 

In [2]:
# Project variables 
#
# These are the project variable used in this ML Model: 
#

PROJECT_ID = "" # @param {type:"string"}
automl_type = "text" # @param {type:"string"}
model_display_name = "automl_text_happiness_model"
job_display_name = "automl_text_happiness_model"
endpoint_display_name = "text_happiness_labelling_endpoint"

## datasets
dataset_display_name = "automl_text_happiness_dataset"
dataset_source_uri = "gs://cloud-ml-data/NL-classification/happiness.csv"
dataset_source_public = "https://storage.googleapis.com/cloud-ml-data/NL-classification/happiness.csv"

# bucket details
BUCKET_NAME = "auto-ml-tutorials"
BUCKET_URI = f"gs://{BUCKET_NAME}/{automl_type}/"
OUTPUTBUCKET = f"gs://auto_ml_datasets_predictions/{automl_type}/"

In [ ]:
! gcloud config set project $PROJECT_ID
print(PROJECT_ID)

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [4]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

print("Region:", REGION)

Region: us-central1


### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

1. **Click Create service account**.

2. In the **Service account name** field, enter a name, and click **Create**.

3. In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex AI" into the filter box, and select **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

4. Click Create. A JSON file that contains your key downloads to your local environment.

5. Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [5]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [6]:
! gsutil ls -al $BUCKET_URI

         0  2022-07-23T20:52:11Z  gs://auto-ml-tutorials/text/#1658609531922655  metageneration=1
   1339288  2022-07-27T18:12:17Z  gs://auto-ml-tutorials/text/happiness.csv#1658945537542803  metageneration=1
TOTAL: 2 objects, 1339288 bytes (1.28 MiB)


### Import libraries and define constants

In [7]:
import jsonlines
from google.cloud import aiplatform, storage

### Initialize Vertex AI 

Initialize the Vertex AI SDK for Python for your project.

In [8]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## Create a `Dataset` resource and import your data

The notebook uses the 'Happy Moments' dataset for demonstration purposes. You can change it to another text classification dataset that [conforms to the data preparation requirements](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text#classification).

Using the Python SDK, you create a dataset and import the dataset in one call to `TextDataset.create()`, as shown in the following cell.

Creating and importing data is a long-running operation. This next step can take a while. The `create()` method waits for the operation to complete, outputting statements as the operation progresses. The statements contain the full name of the dataset that you will use in the following section.

**Note**: You can close the noteboook while you wait for this operation to complete. 

In [20]:
# Create dataset if it doesn't exist

datasets = aiplatform.TextDataset.list(filter = f"display_name={dataset_display_name}")

if datasets:
    text_dataset = datasets[0]
    print(f"Dataset Exists: {datasets[0].resource_name}")
else:
    text_dataset = aiplatform.TextDataset.create(
    display_name=dataset_display_name,
    gcs_source=dataset_source_public,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
    sync=True,
    )
    print(f"Dataset Created: {text_dataset.resource_name}")
    
print(f'Review the Dataset in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/datasets/{datasets[0].resource_name}?project={PROJECT_ID}')



Dataset Exists: projects/993987777814/locations/us-central1/datasets/5227186030586626048
Review the Dataset in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/datasets/projects/993987777814/locations/us-central1/datasets/5227186030586626048?project=paulkamau


In [12]:
# Read the data
dataset_sample = pd.read_csv(dataset_source_public,nrows=5)
                                   
# Use the next code cell to print the first five rows of the data.
dataset_sample.head()

# Shape of training data (num_rows, num_columns)
print(dataset_sample.head())


  I went on a successful date with someone I felt sympathy and connection with.  \
0  I was happy when my son got 90% marks in his e...                              
1       I went to the gym this morning and did yoga.                              
2  We had a serious talk with some friends of our...                              
3  I went with grandchildren to butterfly display...                              
4                            I meditated last night.                              

   affection  
0  affection  
1   exercise  
2    bonding  
3  affection  
4    leisure  


## Train your text classification model


Now you can begin training your model. Training the model is a two part process:

1. **Define the training job.** You must provide a display name and the type of training you want when you define the training job.
2. **Run the training job.** When you run the training job, you need to supply a reference to the dataset to use for training. At this step, you can also configure the data split percentages.

You do not need to specify [data splits](https://cloud.google.com/vertex-ai/docs/general/ml-use). The training job has a default setting of  training 80%/ testing 10%/ validate 10% if you don't provide these values.

To train your model, you call `AutoMLTextTrainingJob.run()` as shown in the following snippets. The method returns a reference to your new `Model` object.

As with importing data into the dataset, training your model can take a substantial amount of time. The client library prints out operation status messages while the training pipeline operation processes. You must wait for the training process to complete before you can get the resource name and ID of your new model, which is required for model evaluation and model deployment.

**Note**: You can close the notebook while you wait for the operation to complete.

In [35]:
# Define the training job and create one if it doesn't exit 
jobs = aiplatform.AutoMLTextTrainingJob.list(filter = f"display_name={job_display_name}")

if jobs:
    job = jobs[0]
    print(f"Jobs Exists: {jobs[0].resource_name}")
else:
    job = aiplatform.AutoMLTextTrainingJob(
    display_name=job_display_name,
    prediction_type="classification",
    multi_label=False,
    )
    print(f"job Created: {job.resource_name}")
    
print(f'Review the job in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/jobs/{jobs[0].resource_name}?project={PROJECT_ID}')


Jobs Exists: projects/993987777814/locations/us-central1/trainingPipelines/5995709199152054272
Review the job in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/jobs/projects/993987777814/locations/us-central1/trainingPipelines/5995709199152054272?project=paulkamau


In [38]:
# Create model job if it doesn't exist
models = aiplatform.Model.list(filter = f"display_name={model_display_name}")

if models:
    model = models[0]
    print(f"Model Exists: {models[0].resource_name}")
else:
    model = job.run(
    dataset=text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.1,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    sync=True,
)
print(f"Model Created: {model.resource_name}")


Model Exists: projects/993987777814/locations/us-central1/models/2888421444214063104
Model Created: projects/993987777814/locations/us-central1/models/2888421444214063104


## Review model evaluation scores

After your model training has finished, you can review the evaluation scores for it using the `list_model_evaluations()` method. This method will return an iterator for each evaluation slice.

In [39]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

{'name': 'projects/993987777814/locations/us-central1/models/2888421444214063104@1/evaluations/739824059654602752', 'metricsSchemaUri': 'gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml', 'metrics': {'confusionMatrix': {'annotationSpecs': [{'displayName': 'exercise', 'id': '175761881502318592'}, {'id': '2049259326488444928', 'displayName': 'achievement'}, {'id': '2481604890716012544', 'displayName': 'leisure'}, {'displayName': 'bonding', 'id': '4355102335702138880'}, {'displayName': 'enjoy_the_moment', 'id': '4787447899929706496'}, {'id': '6660945344915832832', 'displayName': 'nature'}, {'id': '8966788354129526784', 'displayName': 'affection'}], 'rows': [[47.0, 8.0, 3.0, 1.0, 0.0, 1.0, 0.0], [7.0, 1103.0, 25.0, 7.0, 41.0, 6.0, 38.0], [0.0, 33.0, 180.0, 2.0, 28.0, 1.0, 7.0], [0.0, 16.0, 4.0, 403.0, 7.0, 0.0, 22.0], [1.0, 72.0, 26.0, 4.0, 310.0, 8.0, 19.0], [0.0, 6.0, 3.0, 0.0, 6.0, 80.0, 3.0], [1.0, 43.0, 5.0, 8.0, 18.0, 2.0, 1314.0]]}, 'confidenceMe

## Deploy your text classification model to an Endpoint

Once your model has completed training, you must deploy it to an _endpoint_ to get online predictions from it. When you deploy the model to an endpoint, a copy of the model is made on the endpoint with a new resource name and display name.

You can deploy multiple models to the same endpoint and split traffic between the various models assigned to the endpoint. However, you must deploy one model at a time to the endpoint. To change the traffic split percentages, you must assign new values on your second (and subsequent) models each time you deploy a new model.

The following code block demonstrates how to deploy a model. The code snippet relies on the Python SDK to create a new endpoint for deployment. The call to `modely.deploy()` returns a reference to an `Endpoint` object--you need this reference for online predictions in the next section.

In [44]:

# Check if Endpoint Exists 
endpoints = aiplatform.Endpoint.list(filter = f"display_name={endpoint_display_name}")

if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{endpoint_display_name}",
        labels = {'series' : f"{endpoint_display_name}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')


Endpoint Exists: projects/993987777814/locations/us-central1/endpoints/5420706674144968704
Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/5420706674144968704?project=paulkamau


## Get online predictions from your model

Now that you have your endpoint, you can get online predictions from the text classification model. To get the online prediction, you send a prediction request to your endpoint.

In [42]:
content = "I got a high score on my math final!"

response = endpoint.predict(instances=[{"content": content}])

for prediction_ in response.predictions:
    ids = prediction_["ids"]
    display_names = prediction_["displayNames"]
    confidence_scores = prediction_["confidences"]
    for count, id in enumerate(ids):
        print(f"Prediction ID: {id}")
        print(f"Prediction display name: {display_names[count]}")
        print(f"Prediction confidence score: {confidence_scores[count]}")

FailedPrecondition: ignored

## Get batch predictions from your model

You can get batch predictions from a text classification model without deploying it. You must first format all of your prediction instances (prediction input) in JSONL format and you must store the JSONL file in a Google Cloud Storage bucket. You must also provide a Google Cloud Storage bucket to hold your prediction output.

To start, you must first create your predictions input file in JSONL format. Each line in the JSONL document needs to be formatted like so:

```
{ "content": "gs://sourcebucket/datasets/texts/source_text.txt", "mimeType": "text/plain"}
```

The `content` field in the JSON structure must be a Google Cloud Storage URI to another document that contains the text input for prediction.
[See the documentation for more information.](https://cloud.google.com/ai-platform-unified/docs/predictions/batch-predictions#text)

In [ ]:
instances = [
    "We hiked through the woods and up the hill to the ice caves",
    "My kitten is so cute",
]
input_file_name = "automl_text_happiness_dataset_batch_input.jsonl"

For batch prediction, you must supply the following:

+ All of your prediction instances as individual files on Google Cloud Storage, as TXT files for your instances
+ A JSONL file that lists the URIs of all your prediction instances
+ A Cloud Storage bucket to hold the output from batch prediction

For this tutorial, the following cells create a new Storage bucket, upload individual prediction instances as text files to the bucket, and then create the JSONL file with the URIs of your prediction instances.

In [ ]:
# Instantiate the Storage client and create the new bucket
# from google.cloud import  storage
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
# Iterate over the prediction instances, creating a new TXT file
# for each.
input_file_data = []
for count, instance in enumerate(instances):
    instance_name = f"input_{count}.txt"
    instance_file_uri = f"{BUCKET_URI}/{instance_name}"
    # Add the data to store in the JSONL input file.
    tmp_data = {"content": instance_file_uri, "mimeType": "text/plain"}
    input_file_data.append(tmp_data)

    # Create the new instance file
    blob = bucket.blob(instance_name)
    blob.upload_from_string(instance)

input_str = "\n".join([str(d) for d in input_file_data])
file_blob = bucket.blob(f"{input_file_name}")
file_blob.upload_from_string(input_str)

Now that you have the bucket with the prediction instances ready, you can send a batch prediction rhttps://storage.googleapis.com/upload/storage/v1/b/gs://vertex-ai-devaip-20220728004429/o?uploadType=multipartequest to Vertex AI. When you send a request to the service, you must provide the URI of your JSONL file and your output bucket, including the `gs://` protocols.

With the Python SDK, you can create a batch prediction job by calling `Model.batch_predict()`.

In [ ]:
job_display_name = "automl_text_happiness_model_batch_predict_job"

# model = aiplatform.Model(model_name=model.name)
batch_prediction_job = model.batch_predict(
    job_display_name=job_display_name,
    gcs_source=f"{BUCKET_URI}/{input_file_name}",
    gcs_destination_prefix=f"{BUCKET_URI}/output",
    sync=True,
)
batch_prediction_job_name = batch_prediction_job.resource_name

Once the batch prediction job completes, the Python SDK prints out the resource name of the batch prediction job in the format `projects/[PROJECT_ID]/locations/[LOCATION]/batchPredictionJobs/[BATCH_PREDICTION_JOB_ID]`. You can query the Vertex AI service for the status of the batch prediction job using its ID.

The following code snippet demonstrates how to create an instance of the `BatchPredictionJob` class to review its status. Note that you need the full resource name printed out from the Python SDK for this snippet.


## BatchPredictionJob

In [ ]:
from google.cloud.aiplatform import jobs

batch_job = jobs.BatchPredictionJob(batch_prediction_job_name)
print(f"Batch prediction job state: {str(batch_job.state)}")

After the batch job has completed, you can view the results of the job in your output Storage bucket. You might want to first list all of the files in your output bucket to find the URI of the output file.

In [ ]:
BUCKET_OUTPUT = f"{BUCKET_URI}/output"

! gsutil ls -a $BUCKET_OUTPUT

The output from the batch prediction job should be contained in a folder (or _prefix_) that includes the name of the batch prediction job plus a time stamp for when it was created.

For example, if your batch prediction job name is `my-job` and your bucket name is `my-bucket`, the URI of the folder containing your output might look like the following:

```
gs://my-bucket/output/prediction-my-job-2021-06-04T19:54:25.889262Z/
```

To read the batch prediction results, you must download the file locally and open the file. The next cell copies all of the files in the `BUCKET_OUTPUT_FOLDER` into a local folder.

In [ ]:
import os

RESULTS_DIRECTORY = "prediction_results"
RESULTS_DIRECTORY_FULL = f"{RESULTS_DIRECTORY}/output"

# Create missing directories
os.makedirs(RESULTS_DIRECTORY, exist_ok=True)

# Get the Cloud Storage paths for each result
! gsutil -m cp -r $BUCKET_OUTPUT $RESULTS_DIRECTORY

# Get most recently modified directory
latest_directory = max(
    (
        os.path.join(RESULTS_DIRECTORY_FULL, d)
        for d in os.listdir(RESULTS_DIRECTORY_FULL)
    ),
    key=os.path.getmtime,
)

print(f"Local results folder: {latest_directory}")

## JsonLines

With all of the results files downloaded locally, you can open them and read the results. In this tutorial, you use the [`jsonlines`](https://jsonlines.readthedocs.io/en/latest/) library to read the output results.

The following cell opens up the JSONL output file and then prints the predictions for each instance.

In [ ]:
# Get downloaded results in directory
results_files = []
for dirpath, _, files in os.walk(latest_directory):
    for file in files:
        if file.find("predictions") >= 0:
            results_files.append(os.path.join(dirpath, file))


# Consolidate all the results into a list
results = []
for results_file in results_files:
    # Open each result
    with jsonlines.open(results_file) as reader:
        for result in reader.iter(type=dict, skip_invalid=True):
            instance = result["instance"]
            prediction = result["prediction"]
            print(f"\ninstance: {instance['content']}")
            for key, output in prediction.items():
                print(f"\n{key}: {output}")

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

* Dataset
* Training job
* Model
* Endpoint
* Batch prediction
* Batch prediction bucket

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

batch_job.delete()
endpoint.undeploy_all()
# `force` parameter ensures that models are undeployed before deletion
endpoint.delete()

model.delete()

text_dataset.delete()

# Training job
job.delete()